# MATH 441 Optimization Problems

**February 9, 2024**

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp

## Assignment Problem

Suppose there are $n$ agents and $n$ tasks such that the cost to assign agent $i$ to task $j$ is $c_{ij}$. Each agent is assigned a task, and each task is assigned an agent. Let $x_{ij}$ be 1 if we assign agent $i$ to task $j$ and 0 otherwise. Find the assignment matrix $X = [x_{ij}]$ which minimizes the total cost

$$
\sum_{i,j} c_{ij} x_{ij}
$$

The constraints that each agent is assigned to a single task are formulated as:

$$
\sum_{j} x_{ij} = 1 \ , \ \ \text{for all} \ i
$$

The constraints that each task is assigned an agent are formulated as:

$$
\sum_{i} x_{ij} = 1 \ , \ \ \text{for all} \ j
$$

Note: this is just a transportation problem! The agents are supply nodes with supply $s_i = 1$ for each $i$ and the tasks are demand nodes with demand $d_j = 1$ for each $j$. But we have the added integrality constraints $x_{ij} = 0$ or $1$.

## Example: Assigning According to Preference

Students apply for a school house (A,B,C,D) and state their preference by ranking the schools from 1 to 4. Assign students to satisfy the preferences as much as possible. There should be an equal amount of students in each house.

Let $n = 4m$ be the number of total students. There will be $m$ students in each house. Let $A=0$, $B=1$, $C=2$ and $D=3$ be the indices of the houses.

* Decision variables: $x_{ij} = 1$ if student $i$ is assigned house $j$ and $x_{ij} = 0$ otherwise
* Cost matrix: $c_{ij}$ is student $i$'s ranking of house $j$ (so each row of $C$ is a permuation of $(1,2,3,4)$)
* Objective function which is small when preferences are satisfied: $\sum c_{ij} x_{ij}$
* Each student is asssigned 1 house:
    $$
    \sum_{j} x_{ij} = 1 \ , \ \ \text{for all} \ i
    $$
* Each house has ...
    $$
\sum_{i} x_{ij} = m \ , \ \ \text{for all} \ j
$$

In [3]:
import cvxpy as cp
import numpy as np

Create a random matrix of rankings with each house chosen with probability 1/4.

In [4]:
m = 20
n = 4*m

C = np.zeros((n,4))
for i in range(n):
    C[i,:] = np.random.permutation([1.,2.,3.,4.])

In [5]:
C[:5]

array([[1., 3., 4., 2.],
       [2., 3., 1., 4.],
       [4., 1., 2., 3.],
       [4., 3., 2., 1.],
       [1., 4., 2., 3.]])

Define the variables, constraints and objective function.

In [17]:
X = cp.Variable((n,4))
objective = cp.Minimize(cp.sum(cp.multiply(C,X)))
constraint1 = [cp.sum(X[i,:]) == 1 for i in range(n)]
constraint2 = [cp.sum(X[:,j]) == m for j in range(4)]
constraint3 = [X >= 0]
constraints = constraint1 + constraint2 + constraint3
problem = cp.Problem(objective,constraints)

In [18]:
problem.solve()

86.00000003050681

In [19]:
Xs = np.round(X.value,4)

In [20]:
Xs[:5].round(2)

array([[1.  , 0.  , 0.  , 0.  ],
       [0.06, 0.  , 0.94, 0.  ],
       [0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 1.  ],
       [1.  , 0.  , 0.  , 0.  ]])

We phrased the problem as an ordinary LP problem and we got non-integer values in the solution. We need to restrict to $x_{ij}=0,1$. Set `integer=True` in the definition of $X$. The constraints guarantee that $0\leq x_{ij} \leq 1$.

In [21]:
X = cp.Variable((n,4),integer=True)
objective = cp.Minimize(cp.sum(cp.multiply(C,X)))
constraint1 = [cp.sum(X[i,:]) == 1 for i in range(n)]
constraint2 = [cp.sum(X[:,j]) == m for j in range(4)]
constraint3 = [X >= 0]
constraints = constraint1 + constraint2 + constraint3
problem = cp.Problem(objective,constraints)

In [22]:
problem.solve()

86.0

In [24]:
X.value[:10]

array([[ 1., -0., -0., -0.],
       [-0., -0.,  1., -0.],
       [-0.,  1., -0., -0.],
       [-0., -0., -0.,  1.],
       [ 1., -0., -0., -0.],
       [ 1., -0., -0., -0.],
       [-0.,  1., -0., -0.],
       [-0., -0.,  1., -0.],
       [-0.,  1., -0., -0.],
       [-0., -0.,  1., -0.]])